In [1]:
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

In [2]:
query_genes = pd.read_excel("data/1-s2.0-S2211124722010920-mmc2.xlsx") # We only care about the genes from this df
cancer_types = pd.read_excel("data/TCGA-CDR-SupplementalTableS1.xlsx") # And this df maps patients to cancer type
cancer_types = cancer_types.loc[:, ["bcr_patient_barcode", "type"]] 

In [8]:
# pandas cannot read the table because it's too large. So, I will split it into files, 100000 lines each.
Path("data/partitioned_mafs").mkdir(exist_ok=True)
with open("data/mc3.v0.2.8.PUBLIC.maf", "r") as fl_r:
    file = []
    filenumber = 1
    for i in tqdm(range(3600963)): # I counted the number of lines in advance
        line = fl_r.readline().strip()
        if i == 0: # first line is column names, I need to insert it in every file
            colnames = line
        else:
            file.append(line.strip())
        if i % 100000 == 0:
            with open(f"data/partitioned_mafs/maf_{filenumber}.maf", "w") as fl_w:
                fl_w.write(colnames + "\n".join(file))
            file = []
            filenumber += 1
with open(f"data/partitioned_mafs/maf_{filenumber}.maf", "w") as fl:
    fl.write(colnames + "\n".join(file))

  0%|          | 0/3600963 [00:00<?, ?it/s]

In [3]:
# Constructing the variant classification dataframe with gene name and sample code. 
# This one has 3 columns instead of > 100 and only rows with coding mutations, that's why it won't be as heavy. 
dfs = []
substitution_types = []
patients = []
for i in tqdm(range(1, 39)): # just because I know that there are 39 dataframes
    maf = pd.read_csv(f"data/partitioned_mafs/maf_{i}.maf", sep="\t", low_memory=False)
    patients += ["-".join(el.split("-")[:3]) for el in maf.Tumor_Sample_Barcode]
    patients = list(set(patients))
    substitution_types += list(maf.Variant_Classification.unique())
    maf = maf.loc[:, ["Hugo_Symbol", "Tumor_Sample_Barcode", "Variant_Classification"] + ["Start_Position", "End_Position"]]
    maf = maf.loc[maf.Variant_Classification.isin(
        ["Missense_Mutation", "Nonsense_Mutation", "Frame_Shift_Ins", "Frame_Shift_Del"])] # These are coding mutations, I suppose
    dfs.append(maf)
df = pd.concat(dfs)
substitution_types = list(set(substitution_types))

  0%|          | 0/38 [00:00<?, ?it/s]

In [4]:
len(patients)

10224

In [5]:
# Splice sites?
# Just to show to David to make sure I caught all the coding mutations
substitution_types

['Silent',
 'RNA',
 'Intron',
 'In_Frame_Del',
 "5'UTR",
 'Frame_Shift_Ins',
 'Nonsense_Mutation',
 "5'Flank",
 'Missense_Mutation',
 'In_Frame_Ins',
 "3'Flank",
 'Nonstop_Mutation',
 'Frame_Shift_Del',
 'Splice_Site',
 "3'UTR",
 'Translation_Start_Site']

In [6]:
df = df.loc[df.Hugo_Symbol.isin(list(query_genes.gene_name)), :] # Dropping genes we are not interested in
df["bcr_patient_barcode"] = df.Tumor_Sample_Barcode.apply(lambda el: "-".join(el.split("-")[:3])) # Making the format the same as in cancer_types df
final_df = pd.merge(df, cancer_types, on="bcr_patient_barcode") # Adding cancer type column to the data
final_df = final_df.loc[:, ["Hugo_Symbol", "Variant_Classification", "bcr_patient_barcode", "type"]]

In [7]:
# We probably want only genes that were query for this specific cancer type?

query_genes["Hugo_Symbol"] = query_genes["gene_name"]
query_genes["type"] = query_genes["cancer_type"]

final_df = pd.merge(query_genes.loc[:, ["Hugo_Symbol", "type"]].reset_index(), final_df, how='inner').set_index('index')

In [41]:
# create table patients x genes

table = pd.pivot_table(final_df.loc[:, ["Hugo_Symbol", "bcr_patient_barcode"]], index='bcr_patient_barcode', 
                       columns='Hugo_Symbol', aggfunc=lambda x: 1, fill_value=0)
table.reset_index(inplace=True)
# table["bcr_patient_barcode"] = table.index
table["cancer_type"] = table.bcr_patient_barcode.apply(lambda el: list(final_df.loc[final_df.bcr_patient_barcode == el, "type"])[0])
# table.drop("ind", axis=1, inplace=True)

In [44]:
# Add 0 counts to the table

i = 0
for patient in patients:
    if patient not in list(table.bcr_patient_barcode): # If patient is not in the patients x genes table
        if patient in list(cancer_types.bcr_patient_barcode): # But he had at least one of the cancers we are interested in
            dct = {el: [0] for el in list(table.columns)[1:-1]} # Creating a dictionary that I will make into a new line of the table. Line with zeros.
            dct["bcr_patient_barcode"] = [patient]
            ct = list(cancer_types.loc[cancer_types.bcr_patient_barcode == patient, "type"])[0] # ct for cancer type
            dct["cancer_type"] = [ct]
            table = pd.concat([table, pd.DataFrame.from_dict(dct)])
        else:
            i += 1
            print(i, patient)
        

1 TCGA-16-1048
2 TCGA-28-2501
3 TCGA-17-Z036
4 TCGA-17-Z060
5 TCGA-17-Z004
6 TCGA-2G-AALT
7 TCGA-17-Z014
8 TCGA-17-Z010
9 TCGA-17-Z030
10 TCGA-17-Z043
11 TCGA-17-Z022
12 TCGA-17-Z017
13 TCGA-17-Z002
14 TCGA-2G-AALR
15 TCGA-17-Z058
16 TCGA-17-Z011
17 TCGA-17-Z023
18 TCGA-17-Z051
19 TCGA-2G-AALQ
20 TCGA-17-Z044
21 TCGA-17-Z057
22 TCGA-2G-AALF
23 TCGA-2G-AALG
24 TCGA-17-Z050
25 TCGA-17-Z037
26 TCGA-2G-AAM3
27 TCGA-17-Z013
28 TCGA-17-Z027
29 TCGA-17-Z005
30 TCGA-17-Z007
31 TCGA-17-Z032
32 TCGA-17-Z031
33 TCGA-17-Z045
34 TCGA-F5-6810
35 TCGA-36-2539
36 TCGA-17-Z033
37 TCGA-2G-AALO
38 TCGA-17-Z062
39 TCGA-17-Z001
40 TCGA-17-Z053
41 TCGA-2G-AAM4
42 TCGA-17-Z020
43 TCGA-5M-AATA
44 TCGA-17-Z047
45 TCGA-17-Z041
46 TCGA-17-Z026
47 TCGA-17-Z018
48 TCGA-2G-AAKO
49 TCGA-17-Z009
50 TCGA-28-2510
51 TCGA-2G-AALY
52 TCGA-2G-AALX
53 TCGA-17-Z028
54 TCGA-2G-AAM2
55 TCGA-2G-AALW
56 TCGA-17-Z056
57 TCGA-17-Z052
58 TCGA-2G-AALS
59 TCGA-17-Z059
60 TCGA-17-Z048
61 TCGA-17-Z000
62 TCGA-17-Z042
63 TCGA-17-Z025
6

In [46]:
table.to_csv("preprocessed_data_query_genes_by_cancer_type.tsv", sep="\t", index=False)